In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import time
import sys
sys.path.insert(0,'../..')
from keras import losses, metrics
from tensorflow.keras import optimizers
import gp2
from gp2 import Runner

2023-04-25 19:20:57.494499: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-04-25 19:20:59.168091: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-04-25 19:20:59.647724: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0f:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-04-25 19:20:59.650290: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-04-25 19:20:59.652839: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:4e:00.0 name: A100-SXM

In [3]:
help(gp2.KUNetPlus2D)

Help on class KUNetPlus2D in module gp2.gp2.classifiers.k_unet_plus2d:

class KUNetPlus2D(gp2.gp2.classifiers.base_keras_segmentation_classifier.BaseKerasSegmentationClassifier)
 |  KUNetPlus2D(input_size=(512, 512, 1), filter_num=None, n_labels=1, stack_num_down=3, stack_num_up=3, activation='ReLU', output_activation='Sigmoid', batch_norm=True, pool=True, unpool=True, deep_supervision=False, backbone=None, weights='imagenet', freeze_backbone=True, freeze_batch_norm=True, name='xnet', optimizer=None, loss=None, metric=None, verbose=False, workingdir='/tmp')
 |  
 |  Keras U-net++ 2D model.
 |  
 |  Method resolution order:
 |      KUNetPlus2D
 |      gp2.gp2.classifiers.base_keras_segmentation_classifier.BaseKerasSegmentationClassifier
 |      gp2.gp2.classifiers.classifier.Classifier
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, input_size=(512, 512, 1), filter_num=None, n_labels=1, stack_num_down=3, stack_num_up=3, activation='ReLU', o

In [4]:
R = Runner(verbose=True, 
           classifier='kunetplus2d',
           discriminator='cnn',
           filter_num= [16, 32, 64, 128, 256, 512, 1024],
           stack_num_down=3, 
           stack_num_up=3, 
           activation='ReLU', 
           output_activation='Sigmoid', 
           batch_norm=True, 
           pool=True, 
           unpool=True, 
           deep_supervision=False,
           weights=None, 
           freeze_backbone=True, 
           freeze_batch_norm=True,
           optimizer=None, 
           loss=gp2.Util.hybrid_loss, 
           metric=None           
          )

2023-04-25 19:20:59.807910: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 19:21:00.391323: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0f:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-04-25 19:21:00.394840: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-04-25 19:21:00.398344: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 

*** GP2  KUNetPlus2D ***
Working directory: /tmp/tmpdw1cqfvtGP2
Verbose mode active!
{'verbose': True, 'workingdir': '/tmp/tmpdw1cqfvtGP2', 'input_size': (512, 512, 1), 'filter_num': [16, 32, 64, 128, 256, 512, 1024], 'n_labels': 1, 'stack_num_down': 3, 'stack_num_up': 3, 'activation': 'ReLU', 'output_activation': 'Sigmoid', 'batch_norm': True, 'pool': True, 'unpool': True, 'deep_supervision': False, 'backbone': None, 'weights': 'imagenet', 'freeze_backbone': True, 'freeze_batch_norm': True, 'name': 'xnet', 'optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f9b417c5e50>, 'loss': <tensorflow.python.eager.def_function.Function object at 0x7f9b497e9760>, 'metric': [<tensorflow.python.eager.def_function.Function object at 0x7f9b497e94c0>], 'model': <tensorflow.python.keras.engine.functional.Functional object at 0x7f9ae40d5880>}
Model summary:
Model: "xnet_model"
_________________________________________________________________
Layer (type)                 Output Shap

In [5]:
# load our larger toy dataset (10k images and masks)
images = np.load('/hpcstor6/scratch01/r/ryan.zurrin001/GP2TOYEXAMPLE_LARGE/images.npy')
masks = np.load('/hpcstor6/scratch01/r/ryan.zurrin001/GP2TOYEXAMPLE_LARGE/masks.npy')

In [6]:
images = images[:5000]
masks = masks[:5000]

In [7]:
images.shape

(5000, 512, 512, 1)

In [8]:
masks.shape

(5000, 512, 512, 1)

In [9]:
weights = {
    'A': 0.5,
    'A_train': 0.1,
    'A_val': 0.3,
    'A_test': 0.6,
    'B': 0.3,
    'B_train': 0.7,
    'B_val': 0.1,
    'B_test': 0.2,
    'Z': 0.2
}

In [ ]:
R.run(images=images, masks=masks, weights=weights, runs=7)     

Weights OK!
******
Loop 0


2023-04-25 19:21:21.145101: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-04-25 19:21:21.146528: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2245755000 Hz


Epoch 1/100


2023-04-25 19:21:25.689118: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-04-25 19:21:26.309125: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2023-04-25 19:21:26.971433: W tensorflow/stream_executor/gpu/asm_compiler.cc:191] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2023-04-25 19:21:26.971458: W tensorflow/stream_executor/gpu/asm_compiler.cc:194] Used ptxas at ptxas
2023-04-25 19:21:26.971514: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-04-25 19:21:26.987212: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-04-25 19:21:27.742350: I tensorflow/stream_exec

4/4 [==============================] - 37s 6s/step - loss: 1.7666 - dice_coef: 0.1777 - val_loss: 1.7859 - val_dice_coef: 0.1633


2023-04-25 19:22:02.661401: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 2/100
4/4 [==============================] - 4s 1s/step - loss: 1.6877 - dice_coef: 0.2335 - val_loss: 1.7865 - val_dice_coef: 0.1629
Epoch 3/100
4/4 [==============================] - 4s 1s/step - loss: 1.6224 - dice_coef: 0.2777 - val_loss: 1.7874 - val_dice_coef: 0.1623
Epoch 4/100
4/4 [==============================] - 4s 1s/step - loss: 1.5728 - dice_coef: 0.3109 - val_loss: 1.7884 - val_dice_coef: 0.1616
Epoch 5/100
4/4 [==============================] - 4s 1s/step - loss: 1.5331 - dice_coef: 0.3366 - val_loss: 1.7894 - val_dice_coef: 0.1608
Epoch 6/100
4/4 [==============================] - 4s 1s/step - loss: 1.5004 - dice_coef: 0.3574 - val_loss: 1.7903 - val_dice_coef: 0.1602
Epoch 7/100
4/4 [==============================] - 4s 1s/step - loss: 1.4713 - dice_coef: 0.3756 - val_loss: 1.7912 - val_dice_coef: 0.1595
Epoch 8/100
4/4 [==============================] - 4s 1s/step - loss: 1.4426 - dice_coef: 0.3934 - val_loss: 1.7922 - val_dice_coef: 0.1588
Epoch 9/100
4/4 [===

In [ ]:
R.plot()